In [1]:
import importlib
import full_model

In [2]:
full_model.eng = full_model.start_matlab()


Starting MATLAB engine...
-------------------------
Elapsed: 16.22 sec


In [3]:
images_norm = full_model.loadmat("./saved/yaleB.mat")['X']
labels = full_model.loadmat("./saved/yaleB.mat")['Y'].reshape(-1)

## Try various Hyperparameter tuning methods
* [Scikit-Optimize](https://scikit-optimize.github.io/notebooks/hyperparameter-optimization.html)

In [4]:
importlib.reload(full_model)

<module 'full_model' from 'C:\\Users\\aleks_000\\Desktop\\Mentorship\\!GitHub\\full_model.py'>

In [5]:
from skopt.space import Real, Integer

space  = [Real(10**-2, 10**0, "log-uniform", name='lr_pretrain'),
          Real(10**-3, 10**-1, "log-uniform", name='lr'),
          Real(10**0, 10**2, "log-uniform", name='alpha1'),
          Integer(2, 32, name='maxIter1'),
          Real(10**0, 10**2, "log-uniform", name='alpha2'),
          Integer(2, 32, name='maxIter2'),]

In [6]:
from skopt.utils import use_named_args

@use_named_args(space)
def objective(**params):
    try:
        return 1-full_model.run_model(images_norm, labels, seed, ep_prtr, ep, **params)[0]
        #return params['lr_pretrain']*params['lr']*params['alpha1']*params['alpha2']
    except Exception as ex:
        if(type(ex) == KeyError):
            raise ex
        return 1

In [7]:
from skopt.plots import plot_convergence
import time

def optimize(function, iterations, epochs_pretrain, epochs, random_seed):
    start_time = time.time()
    
    # global b/c I couldn't find a better way to directly pass these to the objective function
    global ep_prtr, ep, seed
    ep_prtr = epochs_pretrain
    ep = epochs
    seed = random_seed
    res = function(objective, space, n_calls=iterations, random_state=random_seed, verbose=True, n_jobs=-1)

    plot_convergence(res, yscale="log")
    print("\n------------------")
    print("Best score: {0:.4f}".format(res.fun))
    for i in range(10, iterations+1, 10):
        print("\t{0:d}: {1:.4f}".format(i, min(res.func_vals[0:i])))
    print("Best parameters: {0:}".format(res.x))
    print("Total time elapsed: {0:.2f} sec\n".format(time.time()-start_time))
    return res

In [563]:
m1 = res1.models[-1]
m2 = res1.models[-2]

In [578]:
m1.rng.bytes(10)

c:\program files\anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function rng is deprecated; Attribute rng was deprecated in version 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


b'\xda^!\x07L\x0cu[\x8b\xdf'

In [577]:
m2.rng.bytes(10)

c:\program files\anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function rng is deprecated; Attribute rng was deprecated in version 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


b'\xda^!\x07L\x0cu[\x8b\xdf'

In [573]:
def asdf(result):
    for m in result.models:
        print(id(m))

In [572]:
def helper(result):
    global models, state, set_vars
    if(set_vars):
        result.models[:-1] = models[:-1]
        result.models[-1] = models[-1]
        result.random_state.set_state(state)
        set_vars = False
    asdf(result)

In [557]:
def reload(result, addtl_calls):
    args = result.specs['args']
    args['n_calls'] = addtl_calls
    args['n_random_starts'] = 0
    args['x0'] = result.x_iters
    args['y0'] = result.func_vals
    args['random_state'] = None
    args['callback'] = helper
    global models, state, set_vars
    models = result.models
    state = result.random_state.get_state()
    set_vars = True
    return gp_minimize(**args)

In [ ]:
res1 = optimize(gp_minimize, 12, 1, 1, 0)

In [ ]:
res12 = reload(res1, 4)

In [ ]:
res2 = optimize(gp_minimize, 16, 1, 1, 0)

In [ ]:
from skopt import gp_minimize
# Pro-tip:
# Program taking too long?
# No problem
# ----------
# For a lossless termination, delete the temp folder,
# wait for the code to finish, then create it again
# or
# Deny Admin permissions to temp.mat,
# wait for the code to finish, then remove the restrictions
gp_s1_50 = optimize(gp_minimize, 50, 201, 101, 0)

In [10]:
from skopt import dummy_minimize
dummy_s1_50 = optimize(dummy_minimize, 50, 201, 101, 0)

In [11]:
from skopt import forest_minimize
forest_s1_50 = optimize(forest_minimize, 50, 201, 101, 0)

In [12]:
from skopt import gbrt_minimize
gbrt_s1_50 = optimize(gbrt_minimize, 50, 201, 101, 0)

In [ ]:
from skopt import dump
dump(gp_s1_50, "optims/gp_s1_50.opt")
dump(dummy_s1_50, "optims/dummy_s1_50.opt")
dump(forest_s1_50, "optims/forest_s1_50.opt")
dump(gbrt_s1_50, "optims/gbrt_s1_50.opt")

In [ ]:
from skopt import load
gp_s1_50 = load("optims/gp_s1_50.opt")
dummy_s1_50 = load("optims/dummy_s1_50.opt")
forest_s1_50 = load("optims/forest_s1_50.opt")
gbrt_s1_50 = load("optims/gbrt_s1_50.opt")